---
title: FastAPI notes
description: Notes on FastAPI
date: 2024-09
categories: [Programming]
---

## Basics

- Build APIs based on standard Python type hints
- Automatically generate interactive documentation
- Fast to code, fewer bugs

In [3]:
#! pip install fastapi
#! pip install "uvicorn[standard]"

- If the following contents are in main.py, can run via `uvicorn main:app --reload`.  
    - `main` refers to main.py and app refers to the object inside main.py.
    - `--reload` reloads the page upon changes, to be used during dev, not prod.
- Can see documentation conforming to OpenAPI standard in `http://127.0.0.1:8000/docs`, from which you can use the endpoints!
- `http://127.0.0.1:8000/redoc` returns documentation in alternative format.
- Use `async def` to make the functions non-blocking, enabling other tasks to run concurrently. Useful when function performs I/O-bound operations, such as database queries, file I/O, or network requests, and when need to handle a large number of concurrent requests efficiently.
- Type hints will be validated with Pydantic, so if use a non-int in `/items/{item_id}`, will get an error.
- Order matters: If `read_user_current` is placed *after* `read_user`, will get an error since FastAPI will read functions top-down and try to validate input to be an integer.
- Use `Enums` if path parameter must come from a certain list of values.  If improper parameter is passed, FastAPI will list available values!
- To have paths be read correctly, use `:path` path converter, allowing the parameter to capture the entire path, including slashes.
- `read_animal` without additional parameters will read off animals 0-10.  With additional parameters, can specify which ones we want via *query parameters*, as in http://127.0.0.1:8000/animals/?skip=0&limit=2.  Here, ? denotes start of query parameters and & separates them.  Can also pass optional parameter as http://127.0.0.1:8000/animals/?skip=0&limit=2&optional_param=3, just make sure to specify it as typing.Optional.
- Can pass and use optional parameters as in `read_user_item`.
- Request body is data sent by client to the API and response body is data sent from API to client.  Use Pydantic to specify request body with POST request type.
    - To send a post request, could test it out in /docs or with curl -X POST "http://127.0.0.1:8000/books/" -H "Content-Type: application/json" -d '{
    "name": "The Great Gatsby",
    "author": "F. Scott Fitzgerald",
    "description": "A novel set in the 1920s",
    "price": 10.99
}'
    - Then can go to /books endpoint to see the books printed.


In [6]:
from fastapi import FastAPI
from enum import Enum
import typing as t
from pydantic import BaseModel

app = FastAPI()

@app.get("/") #route/endpoint
def home_page():
    return {"message":"Hello World!"}

@app.get("/items/{item_id}") #item_id is the path parameter
async def read_item(item_id: int):
    return {"item_id":item_id}

@app.get("/users/me") # will not work if placed after, must be before to be valid
async def read_user_current():
    return {"user_id":"Current user"}

@app.get("/users/{user_id}") 
async def read_user(user_id: int):
    return {"user_id":user_id}

class ModelName(str,Enum):
    ALEXNET = 'ALEXNET'
    RESNET = 'RESNET'
    LENET = 'LENET'

@app.get("/models/{model_name}")
async def get_model(model_name: ModelName):
    if model_name == ModelName.ALEXNET:
        return {'model_name':model_name}
    elif model_name.value == "LENET":
        return {'model_name': model_name}
    else:
        return {'model_name':f"You have selected {model_name.value}"}
    
@app.get("files/{file_path:path}")
async def read_file(file_path:str):
    return {"file_path":file_path}

animal_db = [{"animal_name":'cat'},{"animal_name":'llama'},{"animal_name":'alpaca'}]

@app.get("/animals/")
async def read_animal(skip: int=0, limit: int=10, optional_param: t.Optional[int]=None):
    return {"animals": animal_db[skip:skip+limit], "optional_parameter":optional_param}

@app.get("/users/{user_id}/items/{item_id}")
async def read_user_item(
    user_id: int, item_id: int, q: t.Optional[str]=None, short:bool=False
):
    item = {"item_id":item_id, "owner_id":user_id}
    if q:
        item.update({"q":q})
    if not short:
        item.update({'description':'great item with long description'})
    return item

books_db = []
class Book(BaseModel):
    name:str
    author:str
    description:t.Optional[str]
    price:float

@app.post("/books/")
async def create_item(book:Book):
    books_db.append(book)
    return book 
@app.get("/books/")
async def get_books():
    return books_db

### Notes following "Building Data Science Applications with FastAPI" by François Voron Chapter 2: Python specificities -> asyncio

Q: What's the difference between WSGI and ASGI gateways as it pertains to Django and FastAPI?
WSGI (Web Server Gateway Interface) and ASGI (Asynchronous Server Gateway Interface) are two different specifications for Python web servers and applications. They serve as interfaces between web servers and web applications or frameworks. Here’s a detailed comparison of WSGI and ASGI, particularly in the context of Django and FastAPI:

- WSGI (Web Server Gateway Interface)
Synchronous:

WSGI is designed for synchronous web applications. It handles one request at a time per worker, which can lead to inefficiencies when dealing with I/O-bound operations like database queries or external API calls.
Django:

Django is traditionally a WSGI-based framework. It works well for most web applications but can struggle with real-time features like WebSockets or long-polling due to its synchronous nature.
Common WSGI servers for Django include Gunicorn and uWSGI.
Concurrency:

WSGI applications handle concurrency by using multiple worker processes or threads. Each worker handles one request at a time.
Deployment:

WSGI applications are typically deployed using WSGI servers like Gunicorn, uWSGI, or mod_wsgi (for Apache).
- ASGI (Asynchronous Server Gateway Interface)
Asynchronous:

ASGI is designed for asynchronous web applications. It supports both synchronous and asynchronous code, allowing for more efficient handling of I/O-bound operations and real-time features.
FastAPI:

FastAPI is an ASGI-based framework. It is built from the ground up to support asynchronous programming, making it ideal for applications that require high concurrency, real-time communication, or WebSockets.
Common ASGI servers for FastAPI include Uvicorn and Daphne.
Concurrency:

ASGI applications can handle many requests concurrently using asynchronous I/O. This allows for more efficient use of resources, especially for I/O-bound tasks.
Deployment:

ASGI applications are typically deployed using ASGI servers like Uvicorn, Daphne, or Hypercorn.

In [3]:
#!pip install nest_asyncio # run asyncio within Jupyter's already running even loop

In [5]:
# import asyncio
# async def printer(name: str, times: int)->None:
#     for i in range(times):
#         print(name)
#         await asyncio.sleep(1)
# async def main():
#     await asyncio.gather(
#         printer("A",3),
#         printer("B",3)
#     )
# asyncio.run(main())

# adopting code since Jupyter has it's own event loop
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

async def printer(name: str, times: int) -> None:
    for i in range(times):
        print(name)
        await asyncio.sleep(1)

async def main():
    await asyncio.gather(
        printer("A", 3),
        printer("B", 3)
    )

# Await the main coroutine directly
await main()

A
B
A
B
A
B


- asyncio.sleep(1) was added since *writing code in a coroutine doesn't necessarily mean it will not block*.  Computations are blocking!  I/O opps will not block or we could use multiprocessing.

- Path parameters and their validation

In [6]:
from fastapi import FastAPI, Path
app = FastAPI()

@app.get('/license-plates/{license}')
async def get_license_plate(id: int = Path(...,regex=r"^\w{2}-\d{3}-\w{2}")):
    return {"license":license}

/tmp/ipykernel_6460/3090869963.py:5: DeprecationWarning: `regex` has been deprecated, please use `pattern` instead
  async def get_license_plate(id: int = Path(...,regex=r"^\w{2}-\d{3}-\w{2}")):


- In FastAPI, ... above indicate that we don't want a default value.  RegEx validates French license plates like AB-123-CD.

## Notes by Key Topic

### Installation, virtual environment (conda), running, first app

In [ ]:
%% bash
conda create --name fastapi-env python=3.11
conda activate fastapi-env 
pip install fastapi[all]

- If FastAPI app is called app in main file, run as follows: 
`uvicorn main:app --reload`
- Access interactive documentation using `http://127.0.0.1:8000/docs` (using Swagger UI) or `http://127.0.0.1:8000/redoc` (using ReDoc)

### Defining routes with path and query parameters (for user input) and validating requests.

- Defining path parameters: user_id is a path parameter FastAPI will convert to an integer.

In [ ]:
from fastapi import FastAPI
app = FastAPI() 

@app.get("/users/{user_id}")
def read_user(user_id:int):
    return {"user_id":user_id}


- Defining query parameters via function parameters with default values:

In [ ]:
@app.get("/users/")
def read_user(skip:int=0,limit:int=10):
    return {"skip":skip, "limit":limit}

If a user accesses /users/?skip=5&limit=15, FastAPI will return `{"skip":5, "limit":15}`

- Request validation with Pydantic below.  Make the request as follows:
- `curl -X POST "http://127.0.0.1:8000/users/" -H "Content-Type: application/json" -d '{"id":1, "name":"John Smith", "email":"john@example.com"}'
    - `-X POST`: use HTTP method to post data
    - `-H "Content-Type: application/json"`: add HTTP header to the request and specify that the data being sent is in JSON format
    - -d '{"id":1, "name":"John Smith", "email":"john@example.com"}': send the specified data in the request body


In [ ]:
from pydantic import BaseModel
class User(BaseModel):
    id: int
    name: str
    email: str

@app.post("/users/")
def create_user(user:User):
    return {"id": user.id, "name": user.name, "email":user.email}

- Combining path ahd query parameters with Pydantic: user_id is a path parameter and details is a query parameter that modifies the response.
- Read simply as `curl "http://127.0.0.1:8000/users/1"`

In [ ]:
@app.get("/users/{user_id}")
def read_user(user_id: int, details: bool=False):
    if details:
        return {"user_id":user_id, "details":"Detailed info"}
    return {"user_id":user_id}

### Request and response models
- Request models define the structure of the data that your API expects to receive in the request body. They are used to validate and parse the incoming data.
- Response models define the structure of the data that your API returns in response.  They ensure that the response data is correctly formatted and validated.
- `curl -X POST "http://127.0.0.1:8000/users/" -H "Content-Type: application/json" -d '{"id":1, "name":"John Smith", "email":"john@example.com", "age":30}'`

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import Optional

app = FastAPI()

class UserCreate(BaseModel):
    id: int
    name: str
    email: str
    age: Optional[int] = None

class UserResponse(BaseModel):
    id: int
    name: str
    email: str
    age: Optional[int] = None
    is_active:  bool

@app.post("/users/", response_model=UserResponse)
async def create_user(user:UserCreate): #validate incoming data
    user_response = UserResponse(       #validate outgoing data
        id= user.id,
        name=user.name,
        email=user.email,
        age=user.age,
        is_active=True
    )
    return user_response


### Dependency Injection
- Inject dependencies (database connections, configuration settings, other shared resources) into your functions or classes.
- Separate concerns between the logic of the endpoint and the more generic logic for the pagination parameters. 
- Ideal for utility logic to retrieve or validate data, make security checks, or call external logic that will be needed several times across the application.

- Notes following "Building Data Science Applications with FastAPI" by François Voron Chapter 2: Python specificities -> asyncio

In [ ]:
from fastapi import Depends, FastAPI

app = FastAPI()

async def pagination(skip:int=0,limit:int=10)->tuple[int,int]:
    return (skip,limit)

@app.get("/items")
async def list_items(p:tuple[int,int]=Depends(pagination)):
    skip,limit = p
    return {"skip":skip, "limit":limit}

@app.get("/things")
async def list_things(p:tuple[int,int]=Depends(pagination)):
    skip,limit = p
    return {"skip":skip, "limit":limit}

- FastAPI limitation: `Depends` function is not able to forward the type of the dependency function, so we have to do this manually above.
- Raising a 404 error:

In [ ]:
from fastapi import Depends, FastAPI, HTTPException, status
from pydantic import BaseModel


class Post(BaseModel):
    id: int
    title: str
    content: str


class PostUpdate(BaseModel):
    title: str | None
    content: str | None


class DummyDatabase:
    posts: dict[int, Post] = {}


db = DummyDatabase()
db.posts = {
    1: Post(id=1, title="Post 1", content="Content 1"),
    2: Post(id=2, title="Post 2", content="Content 2"),
    3: Post(id=3, title="Post 3", content="Content 3"),
}


app = FastAPI()


async def get_post_or_404(id: int) -> Post:
    try:
        return db.posts[id]
    except KeyError:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND)


@app.get("/posts/{id}")
async def get(post: Post = Depends(get_post_or_404)):
    return post


@app.patch("/posts/{id}")
async def update(post_update: PostUpdate, post: Post = Depends(get_post_or_404)):
    updated_post = post.copy(update=post_update.dict())
    db.posts[post.id] = updated_post
    return updated_post


@app.delete("/posts/{id}", status_code=status.HTTP_204_NO_CONTENT)
async def delete(post: Post = Depends(get_post_or_404)):
    db.posts.pop(post.id)

#### Creating and using a parametrized dependency with a class
- Suppose we wanted to dynamically cap the limit value in the pagination example...-> would need to do this with a class!

In [ ]:
from fastapi import Depends, FastAPI, Query

app = FastAPI()

class Pagination:
    def __init__(self, maximum_limit:int = 100):
        self.maximum_limit = maximum_limit
    async def __call__(
            self,
            skip: int = Query(0, ge=0),
            limit: int = Query(10,ge=0)
    ) -> tuple[int,int]:
        capped_limit = min(self.maximum_limit, limit)
        return (skip, capped_limit)
# hardcoded below, but could come from config file or env variable
pagination = Pagination(maximum_limit=50)

@app.get("/items")
async def list_items(p: tuple[int, int] = Depends(pagination)):
    skip, limit = p
    return {"skip": skip, "limit": limit}


@app.get("/things")
async def list_things(p: tuple[int, int] = Depends(pagination)):
    skip, limit = p
    return {"skip": skip, "limit": limit}

- Note: n FastAPI, Query is used to define and validate query parameters for your API endpoints. Query parameters are the key-value pairs that appear after the ? in a URL. They are typically used to filter, sort, or paginate data.

- `Depends` simply expects a callable: in can be `__call__` or another function as below.  Note that the pattern below could be used to apply different preprocessing steps, depending on the data, in the ML context:

In [ ]:
from fastapi import Depends, FastAPI, Query

app = FastAPI()


class Pagination:
    def __init__(self, maximum_limit: int = 100):
        self.maximum_limit = maximum_limit

    async def skip_limit(
        self,
        skip: int = Query(0, ge=0),
        limit: int = Query(10, ge=0),
    ) -> tuple[int, int]:
        capped_limit = min(self.maximum_limit, limit)
        return (skip, capped_limit)

    async def page_size(
        self,
        page: int = Query(1, ge=1),
        size: int = Query(10, ge=0),
    ) -> tuple[int, int]:
        capped_size = min(self.maximum_limit, size)
        return (page, capped_size)


pagination = Pagination(maximum_limit=50)


@app.get("/items")
async def list_items(p: tuple[int, int] = Depends(pagination.skip_limit)):
    skip, limit = p
    return {"skip": skip, "limit": limit}


@app.get("/things")
async def list_things(p: tuple[int, int] = Depends(pagination.page_size)):
    page, size = p
    return {"page": page, "size": size}

- Using dependency injection to manage db connection, ensuring that each request gets a fresh connection and that connections are properly closed after use:

In [ ]:
from fastapi import FastAPI, Depends
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# Define a User model
class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    email = Column(String, unique=True, index=True)

app = FastAPI()

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()
# Endpoint to create a new user
@app.post("/users/")
async def create_user(name: str, email: str, db: Session = Depends(get_db)):
    user = User(name=name, email=email)
    db.add(user)
    db.commit()
    db.refresh(user)
    return user

@app.get("/users/")
async def read_users(db: Session=Depends(get_db)):
    users = db.query(User).all()
    return users

# Run the application
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)


- An example in LLM context:

In [ ]:
from fastapi import FastAPI, Depends
from pydantic import BaseModel
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

app = FastAPI()

class LLM:
    def __init__(self, model_name: str = "distilbert-base-uncased-finetuned-sst-2-english"):
        # Load the pre-trained model and tokenizer from Hugging Face
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.model.to("cuda" if torch.cuda.is_available() else "cpu")

    def predict(self, text: str, preprocess_steps: list):
        # Preprocess the text
        text = self.text_processor.preprocess(text, preprocess_steps)
        # Tokenize the input text
        inputs = self.tokenizer(text, return_tensors="pt")
        inputs = {k: v.to("cuda" if torch.cuda.is_available() else "cpu") for k, v in inputs.items()}
        # Perform prediction using the model
        with torch.no_grad():
            outputs = self.model(**inputs)
        # Get the predicted class
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(predictions, dim=1).item()
        return predicted_class

# Create a global LLM instance
llm_instance = LLM()

# Dependency to get the global LLM instance
def get_llm():
    return llm_instance

# Request model for prediction
class PredictionRequest(BaseModel):
    text: str

# Response model for prediction
class PredictionResponse(BaseModel):
    prediction: int

# Use the LLM dependency in an endpoint
@app.post("/predict/", response_model=PredictionResponse)
async def predict(request: PredictionRequest, llm: LLM = Depends(get_llm)):
    prediction = llm.predict(request.text)
    return {"prediction": prediction}

# Run the application
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)

- Run as follows: `curl -X POST "http://127.0.0.1:8000/predict/" -H "Content-Type: application/json" -d '{"text": "I love FastAPI!"}'`
- Will return something like the following: `{
    "prediction": 1
}`